In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow_addons.layers import WeightNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# set seed
def seed_everything(seed = 42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(seed = 42)

In [ ]:
# reading in all the data files
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
train_targets_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
sample_submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
ind_te = test_features[test_features['cp_type']=='ctl_vehicle'].index

In [ ]:
def preprocess(df):
    d = df.copy()
    # change cp_dose: D1 -> 0, D2 -> 1
    d['cp_dose'] = d['cp_dose'].map({'D1':0, 'D2':1})
    # change cp_time: 24 -> 0, 48 -> 1, 72 -> 2
    d['cp_time'] = d['cp_time']//24-1
    # change cp_type: trt_cp -> 1, ctl_vehicle -> 0
    d['cp_type'] = d['cp_type'].map({'ctl_vehicle':0, 'trt_cp':1})
    # drop the sig_id column
    d.drop(columns = ['sig_id'], inplace = True)
    return d

In [ ]:
X_train = preprocess(train_features)
X_test = preprocess(test_features)

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
y_train = train_targets_scored.copy()
# drop the sig_id column
y_train.drop(columns = ['sig_id'], inplace = True)
y_train.shape

In [ ]:
# Stop training when the validation log loss metric has stopped decreasing for 3 epochs.
def callbacks():
    early_stopping = EarlyStopping(monitor = 'val_logloss',
                                   patience = 3,
                                   mode = 'min',
                                   restore_best_weights = True)
    return early_stopping

In [ ]:
# create the neural network model
def create_model(num_columns):
    model = Sequential()
    model.add(Input(num_columns))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(WeightNormalization(Dense(2048, activation="relu")))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(WeightNormalization(Dense(1024, activation="relu")))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(WeightNormalization(Dense(512, activation="relu")))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(WeightNormalization(Dense(206, activation="sigmoid")))
    model.compile(optimizer = Adam(), loss = 'binary_crossentropy')
    return model

In [ ]:
# feats = [0,1,2,.......,873]
feats = np.arange(0,X_train.shape[1],1)
inp_size = 874

In [ ]:
# creating a sample df to store predicted values
sample = sample_submission.copy()

In [ ]:
res = y_train.copy()
sample.loc[:, y_train.columns] = 0
res.loc[:, y_train.columns] = 0
print(sample.shape, res.shape)

In [ ]:
from time import time
import datetime

In [ ]:
def log_loss_metric(y_true, y_pred):
    metrics = []
    for _target in y_train.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels = [0,1]))
    return np.mean(metrics)

In [ ]:
test_preds = []
res_preds = []
np.random.seed(seed=42)
n_splits = 5
n_top = 10
n_round = 1

for seed in range(n_round):
    split_cols = np.random.choice(feats, inp_size, replace = False)
    res.loc[:, y_train.columns] = 0
    sample.loc[:, y_train.columns] = 0
    for n, (tr, te) in enumerate(KFold(n_splits = n_splits, random_state = seed, shuffle = True).split(X_train, y_train)):
        start_time = time()
        x_tr = X_train.values[tr][:, split_cols]
        x_val = X_train.values[te][:, split_cols]
        y_tr, y_val = y_train.values[tr], y_train.values[te]
        x_tt = X_test.values[:, split_cols]   
        for num in range(n_top):
            model = create_model(inp_size)
            model.fit(x_tr, y_tr,validation_data=(x_val, y_val), epochs = 25, batch_size = 128,
                      callbacks = callbacks(), verbose = 0)
            sample.loc[:, y_train.columns] += model.predict(x_tt, batch_size = 128)/(n_splits*n_top)
            res.loc[te, y_train.columns] += model.predict(x_val, batch_size = 128)/(n_top)
        loss = log_loss_metric(y_train.loc[te,y_train.columns], res.loc[te, y_train.columns])
        print(f'[{str(datetime.timedelta(seconds = time() - start_time))[2:7]}], Seed {seed}, Fold {n}:', loss)
        K.clear_session()
        del model
    # set prediction 0 for ctl_vehicle observations
    sample.loc[ind_te, y_train.columns] = 0
    test_preds.append(sample.copy())    
    res_preds.append(res.copy())


In [ ]:
print(len(res_preds), len(test_preds))

In [ ]:
aa = [1.0]
sample.loc[:, y_train.columns] = 0
for i in range(n_round):
    sample.loc[:, y_train.columns] += aa[i] * test_preds[i].loc[:, y_train.columns]
sample.loc[ind_te, y_train.columns] = 0

In [ ]:
display(sample.head())
# write sample to submission.csv file
sample.to_csv('submission.csv', index=False)